In [1]:
import pandas as pd
from youtube_transcript_api import YouTubeTranscriptApi

pd.set_option('display.width', 10000)

url = "https://www.youtube.com/watch?v=wDAmezoNHJY"

vid_id = url.partition("/watch?v=")[2]

#retrieve auto-generated transcripts
transcript_list = YouTubeTranscriptApi.list_transcripts(vid_id)
#transcript = transcript_list.find_manually_created_transcript(['fr', 'fr-FR'])  
transcript = transcript_list.find_generated_transcript(['fr', 'fr-FR']) 
if transcript != "":
    df = pd.DataFrame(transcript.fetch())
    #df.to_csv("transcript.csv", index=False)

In [2]:
from os import path
from yt_dlp import YoutubeDL
from pydub import AudioSegment
from subprocess import STDOUT, DEVNULL, call, run

# Fetch best quality audio, re-encode to mono, 256kbps 16bit wav
# check file bitrate etc... with: ffprobe audio.wav

def fetch_encode_wav(url):
    video_info = YoutubeDL().extract_info(
        url = url,download=False
    )

    options={
        'format':'bestaudio/best',
        'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav',
        }],
        'postprocessor_args': [
            '-ar', '16000',
            '-ac', '1',
        ],
        'prefer_ffmpeg': True,
        'keepvideo':False,
        'outtmpl':'audio/audio.wav',
    }

    with YoutubeDL(options) as ydl:
        ydl.download([video_info['webpage_url']])

    print("Download complete !")

fetch_encode_wav("https://www.youtube.com/watch?v=wDAmezoNHJY")

[youtube] wDAmezoNHJY: Downloading webpage
[youtube] wDAmezoNHJY: Downloading android player API JSON
[youtube] wDAmezoNHJY: Downloading webpage
[youtube] wDAmezoNHJY: Downloading android player API JSON
[info] wDAmezoNHJY: Downloading 1 format(s): 251
[download] audio/audio.wav has already been downloaded
[download] 100% of 22.79MiB
[ExtractAudio] Destination: audio/audio.wav
Deleting original file audio/audio.orig.wav (pass -k to keep)
Download complete !


In [3]:
from subprocess import run
from pydub import AudioSegment, silence
import numpy as np
import pandas as pd
import soundfile as sf
from pydub import AudioSegment
from pydub.playback import play
import os.path

pydub_sample_rate = 16000

# Keep only clips containing specific word

pd.set_option('display.width', 10000)

#df = pd.read_csv("transcript.csv")
df.insert(2, 'stop', df['start'] + df['duration'], False)

# Select rows with matching word regex
word = "photo"
df2 = df[df['text'].str.contains(r'\b'+word+r'\b', regex=True)]
print(f"{df2.shape[0]} occurrence(s) of word : {word}")
#print(df2)
# Make audio, generate SRT

def cut_merge(cuts, audio_in, audio_out):
    # Takes a list of tuples as cuts [(start, end)...]

    cuts_old = len(cuts) 
    for i in range(len(cuts)):

        if i < len(cuts)-1 and cuts[i][1] == cuts[i+1][0]:
            cuts[i+1] = (cuts[i][0], cuts[i+1][1])
            cuts.pop(i)

    print(f"total cuts : {len(cuts)}, removed {cuts_old - len(cuts)} useless cut(s)") 

    # Opening file and extracting segment
    segment = AudioSegment.from_file(audio_in, format="wav", frame_rate=pydub_sample_rate, channels=1)
    clips = []

    for cut in cuts:
        # cut_start, cut_end should be expressed in ms
        clips.append(segment[cut[0]*1000:cut[1]*1000])

    merged_clips = sum(clips)
    merged_clips.export(audio_out, format="wav", parameters=["-ar", str(pydub_sample_rate), "-ac", "1"])

# def rm_silence(audio_in, audio_out):
#     # Audio files should be normalized before removing silence, else manually set top_db everytime ???????

#     # Define a function to normalize a chunk to a target amplitude.
#     def match_target_amplitude(aChunk, target_dBFS):
#         ''' Normalize given audio chunk '''
#         change_in_dBFS = target_dBFS - aChunk.dBFS
#         return aChunk.apply_gain(change_in_dBFS)

#     # Split track where the silence is 2 seconds or more and get chunks using 
#     # the imported function.
#     chunks = silence.split_on_silence (
#         AudioSegment.from_wav(audio_in, frame_rate=16000, channels=1), 
#         # Specify that a silent chunk must be at least 500ms long.
#         min_silence_len = 500,
#         # Consider a chunk silent if it's quieter than -30 dBFS.
#         # (You may want to adjust this parameter.)
#         silence_thresh = -30
#     )

#     #print(chunks)
#     print("CHONKS :")
#     print(len(chunks))

#     silence_length = 1000
#     segments = []
#     for i, chunk in enumerate(chunks):
#         # Create a silence chunk of silence_length ms for smoother transitions (optional)
#         silence_chunk = AudioSegment.silent(duration=silence_length)

#         # Add silence chunk.
#         audio_chunk = silence_chunk + chunk + silence_chunk

#         # Normalize the entire chunk.
#         normalized_chunk = match_target_amplitude(audio_chunk, -20.0)
#         segments.append(normalized_chunk)
    
#     merged_chunks = sum(segments)
#     merged_chunks.export(audio_out, format="wav", frame_rate=16000, channels=1)


cut_merge([(start_time,stop_time) for start_time,stop_time in zip(df2['start'], df2["stop"])], "audio/audio.wav", "audio/audio_clip.wav")
#rm_silence("audio/audio_clip.wav", "audio/audio_clip_unsilenced.wav")

26 occurrence(s) of word : photo
total cuts : 25, removed 1 useless cut(s)


In [4]:
# Use vosk to get words timestamp

from vosk import Model, KaldiRecognizer, SetLogLevel
import sys
import os
import wave
import subprocess
import srt
import json
import datetime

SetLogLevel(-1)

if not os.path.exists("model"):
    print ("Please download the model from https://alphacephei.com/vosk/models and unpack as 'model' in the current folder.")
    exit (1)

sample_rate=16000
model = Model("model")
rec = KaldiRecognizer(model, sample_rate)
rec.SetWords(True)
audio_clip = "audio/audio_clip.wav"

process = subprocess.Popen(['ffmpeg', '-loglevel', 'quiet', '-i',
                            audio_clip,
                            '-ar', str(sample_rate) , '-ac', '1', '-f', 's16le', '-'],
                            stdout=subprocess.PIPE)

WORDS_PER_LINE = 1

def transcribe():
    results = []
    subs = []
    while True:
       data = process.stdout.read(4000)
       if len(data) == 0:
           break
       if rec.AcceptWaveform(data):
           results.append(rec.Result())
    results.append(rec.FinalResult())

    for i, res in enumerate(results):
       jres = json.loads(res)
       if not 'result' in jres:
           continue
       words = jres['result']
       for j in range(0, len(words), WORDS_PER_LINE):
           line = words[j : j + WORDS_PER_LINE] 
           s = srt.Subtitle(index=len(subs), 
                   content=" ".join([l['word'] for l in line]),
                   start=line[0]['start'], 
                   end=line[-1]['end'])
           subs.append(s)
    return subs

words = transcribe()

In [5]:
labels = []
for word in words:
    if word.content == "photo":
        labels.append((word.start, word.end))
       
cut_merge(labels, "audio/audio_clip.wav", "audio/audio_final.wav")
print(f"{len(labels)} words transcribed !")
print("playing final audio, turn volume up...")
final_segment = AudioSegment.from_wav("audio/audio_final.wav")
play(final_segment)

total cuts : 23, removed 0 useless cut(s)
23 words transcribed !
playing final audio, turn volume up...


ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4745:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4745:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4745:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5233:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM sysdefault
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4745:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4745:(_snd_config_evaluate) function snd_func_concat returned error

In [6]:
# AVERAGE RUN TIME ~ 50s for (18-21)/26, 30 % error (transcribed/occurences)

# Roughly 1 10min vid per minute, 60 vids / hours, if vid = 10min and transcribed = 5 : 60*5 = 300 new words per hour
# Worse results when transcribing with unsilenced clip
# Still other vosk models to try !